# Import Dependencies

In [1]:
# import dependencies
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
from pymongo import MongoClient
from pprint import pprint
import joblib

# Load Data

In [2]:
# I will name the dataset data when it is loaded in
df = pd.read_csv("data_files/Model_Ready.csv")
df.head()

,_id,_id_game,game_id,date_start,visitor_name,visitor_nickname,visitor_code,home_id,home_name,home_nickname,...,home_defReb,home_totReb,home_assists,home_pFouls,home_steals,home_turnovers,home_blocks,home_plusMinus,Over/Under,Result
0,664f7fde36cf90b9bbe8fbda,664f7ee136cf90b9bbe8c333,8787,2021-10-19T23:30:00.000Z,Brooklyn Nets,Nets,BKN,21,Milwaukee Bucks,Bucks,...,41,54,25,19,8,7,9,23,240.5,Under
1,664f7fde36cf90b9bbe8fbdb,664f7ee136cf90b9bbe8c334,8788,2021-10-20T02:00:00.000Z,Golden State Warriors,Warriors,GSW,17,Los Angeles Lakers,Lakers,...,40,45,21,25,7,17,4,-7,229.0,Over
2,664f7fde36cf90b9bbe8fbdc,664f7ee136cf90b9bbe8c335,8789,2021-10-20T23:00:00.000Z,Indiana Pacers,Pacers,IND,5,Charlotte Hornets,Hornets,...,34,46,29,21,9,8,5,1,228.5,Over
3,664f7fde36cf90b9bbe8fbdd,664f7ee136cf90b9bbe8c336,8790,2021-10-20T23:00:00.000Z,Chicago Bulls,Bulls,CHI,10,Detroit Pistons,Pistons,...,36,47,17,16,7,16,5,-6,218.0,Under
4,664f7fde36cf90b9bbe8fbde,664f7ee136cf90b9bbe8c337,8791,2021-10-20T23:30:00.000Z,Boston Celtics,Celtics,BOS,24,New York Knicks,Knicks,...,48,55,27,22,9,17,10,4,219.5,Over


# Prepare Data for Encoding

In [3]:
df.columns

Index(['_id', '_id_game', 'game_id', 'date_start', 'visitor_name',
       'visitor_nickname', 'visitor_code', 'home_id', 'home_name',
       'home_nickname', 'home_code', 'visitor_points_game', 'home_points_game',
       '_id_stats', 'visitor_points_stats', 'visitor_fgm', 'visitor_fga',
       'visitor_fgp', 'visitor_ftm', 'visitor_fta', 'visitor_ftp',
       'visitor_tpm', 'visitor_tpa', 'visitor_tpp', 'visitor_offReb',
       'visitor_defReb', 'visitor_totReb', 'visitor_assists', 'visitor_pFouls',
       'visitor_steals', 'visitor_turnovers', 'visitor_blocks',
       'visitor_plusMinus', 'home_points_stats', 'home_fgm', 'home_fga',
       'home_fgp', 'home_ftm', 'home_fta', 'home_ftp', 'home_tpm', 'home_tpa',
       'home_tpp', 'home_offReb', 'home_defReb', 'home_totReb', 'home_assists',
       'home_pFouls', 'home_steals', 'home_turnovers', 'home_blocks',
       'home_plusMinus', 'Over/Under', 'Result'],
      dtype='object')

In [4]:
newdf = df[['home_name','visitor_name','home_points_game','visitor_points_game','home_totReb','visitor_totReb','home_assists','visitor_assists','home_blocks','visitor_blocks','Over/Under','Result']]

In [5]:
newdf.rename(columns={'Over/Under':'over_under'})

,home_name,visitor_name,home_points_game,visitor_points_game,home_totReb,visitor_totReb,home_assists,visitor_assists,home_blocks,visitor_blocks,over_under,Result
0,Milwaukee Bucks,Brooklyn Nets,127,104,54,44,25,19,9,9,240.5,Under
1,Los Angeles Lakers,Golden State Warriors,114,121,45,50,21,30,4,2,229.0,Over
2,Charlotte Hornets,Indiana Pacers,123,122,46,51,29,29,5,10,228.5,Over
3,Detroit Pistons,Chicago Bulls,88,94,47,48,17,18,5,5,218.0,Under
4,New York Knicks,Boston Celtics,138,134,55,56,27,34,10,9,219.5,Over
...,...,...,...,...,...,...,...,...,...,...,...,...
2120,Boston Celtics,Cleveland Cavaliers,120,95,55,38,22,19,7,3,208.5,Over
2121,Boston Celtics,Cleveland Cavaliers,94,118,31,44,22,24,3,3,212.0,Under
2122,Cleveland Cavaliers,Boston Celtics,93,106,35,44,17,22,2,4,211.0,Under
2123,Cleveland Cavaliers,Boston Celtics,102,109,32,48,26,15,3,4,208.0,Over


In [6]:
newdf['total_score'] = newdf['home_points_game'] + newdf['visitor_points_game']

C:\Users\Latasha Jones\AppData\Local\Temp\ipykernel_29776\1845410676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['total_score'] = newdf['home_points_game'] + newdf['visitor_points_game']


In [7]:
newdf.count()

home_name              2125
visitor_name           2125
home_points_game       2125
visitor_points_game    2125
home_totReb            2125
visitor_totReb         2125
home_assists           2125
visitor_assists        2125
home_blocks            2125
visitor_blocks         2125
Over/Under             2125
Result                 2125
total_score            2125
dtype: int64

In [8]:
# Updating target column to True/False values
newdf['Result'].replace('Under',0,inplace=True)
newdf['Result'].replace('Over',1,inplace=True)

C:\Users\Latasha Jones\AppData\Local\Temp\ipykernel_29776\2546420923.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['Result'].replace('Under',0,inplace=True)
C:\Users\Latasha Jones\AppData\Local\Temp\ipykernel_29776\2546420923.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['Result'].replace('Over',1,inplace=True)


In [9]:
newdf

,home_name,visitor_name,home_points_game,visitor_points_game,home_totReb,visitor_totReb,home_assists,visitor_assists,home_blocks,visitor_blocks,Over/Under,Result,total_score
0,Milwaukee Bucks,Brooklyn Nets,127,104,54,44,25,19,9,9,240.5,0,231
1,Los Angeles Lakers,Golden State Warriors,114,121,45,50,21,30,4,2,229.0,1,235
2,Charlotte Hornets,Indiana Pacers,123,122,46,51,29,29,5,10,228.5,1,245
3,Detroit Pistons,Chicago Bulls,88,94,47,48,17,18,5,5,218.0,0,182
4,New York Knicks,Boston Celtics,138,134,55,56,27,34,10,9,219.5,1,272
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2120,Boston Celtics,Cleveland Cavaliers,120,95,55,38,22,19,7,3,208.5,1,215
2121,Boston Celtics,Cleveland Cavaliers,94,118,31,44,22,24,3,3,212.0,0,212
2122,Cleveland Cavaliers,Boston Celtics,93,106,35,44,17,22,2,4,211.0,0,199
2123,Cleveland Cavaliers,Boston Celtics,102,109,32,48,26,15,3,4,208.0,1,211


# Prepocess and Encode Data

In [10]:
# encode the columns (run this column only if needed)
dummies = pd.get_dummies(newdf, columns=['home_name','visitor_name'])
dummies.head()

,home_points_game,visitor_points_game,home_totReb,visitor_totReb,home_assists,visitor_assists,home_blocks,visitor_blocks,Over/Under,Result,...,visitor_name_Oklahoma City Thunder,visitor_name_Orlando Magic,visitor_name_Philadelphia 76ers,visitor_name_Phoenix Suns,visitor_name_Portland Trail Blazers,visitor_name_Sacramento Kings,visitor_name_San Antonio Spurs,visitor_name_Toronto Raptors,visitor_name_Utah Jazz,visitor_name_Washington Wizards
0,127,104,54,44,25,19,9,9,240.5,0,...,False,False,False,False,False,False,False,False,False,False
1,114,121,45,50,21,30,4,2,229.0,1,...,False,False,False,False,False,False,False,False,False,False
2,123,122,46,51,29,29,5,10,228.5,1,...,False,False,False,False,False,False,False,False,False,False
3,88,94,47,48,17,18,5,5,218.0,0,...,False,False,False,False,False,False,False,False,False,False
4,138,134,55,56,27,34,10,9,219.5,1,...,False,False,False,False,False,False,False,False,False,False


In [11]:
X = dummies.copy()
X.drop("Result",axis=1,inplace=True)
X.head()

,home_points_game,visitor_points_game,home_totReb,visitor_totReb,home_assists,visitor_assists,home_blocks,visitor_blocks,Over/Under,total_score,...,visitor_name_Oklahoma City Thunder,visitor_name_Orlando Magic,visitor_name_Philadelphia 76ers,visitor_name_Phoenix Suns,visitor_name_Portland Trail Blazers,visitor_name_Sacramento Kings,visitor_name_San Antonio Spurs,visitor_name_Toronto Raptors,visitor_name_Utah Jazz,visitor_name_Washington Wizards
0,127,104,54,44,25,19,9,9,240.5,231,...,False,False,False,False,False,False,False,False,False,False
1,114,121,45,50,21,30,4,2,229.0,235,...,False,False,False,False,False,False,False,False,False,False
2,123,122,46,51,29,29,5,10,228.5,245,...,False,False,False,False,False,False,False,False,False,False
3,88,94,47,48,17,18,5,5,218.0,182,...,False,False,False,False,False,False,False,False,False,False
4,138,134,55,56,27,34,10,9,219.5,272,...,False,False,False,False,False,False,False,False,False,False


In [12]:
X.columns

Index(['home_points_game', 'visitor_points_game', 'home_totReb',
       'visitor_totReb', 'home_assists', 'visitor_assists', 'home_blocks',
       'visitor_blocks', 'Over/Under', 'total_score',
       'home_name_Atlanta Hawks', 'home_name_Boston Celtics',
       'home_name_Brooklyn Nets', 'home_name_Charlotte Hornets',
       'home_name_Chicago Bulls', 'home_name_Cleveland Cavaliers',
       'home_name_Dallas Mavericks', 'home_name_Denver Nuggets',
       'home_name_Detroit Pistons', 'home_name_Golden State Warriors',
       'home_name_Houston Rockets', 'home_name_Indiana Pacers',
       'home_name_LA Clippers', 'home_name_Los Angeles Lakers',
       'home_name_Memphis Grizzlies', 'home_name_Miami Heat',
       'home_name_Milwaukee Bucks', 'home_name_Minnesota Timberwolves',
       'home_name_New Orleans Pelicans', 'home_name_New York Knicks',
       'home_name_Oklahoma City Thunder', 'home_name_Orlando Magic',
       'home_name_Philadelphia 76ers', 'home_name_Phoenix Suns',
       'ho

In [13]:
y = dummies['Result'].values.reshape(-1,1)

# Split, Train, Scale Data

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [16]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [17]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fit the Random Forest Model

In [18]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=1000, random_state=78)

In [19]:
# Fit the model and use .ravel()on the "y_train" data. 
rf_model = rf_model.fit(X_train_scaled, y_train.ravel())

# Make Predictions Using the Random Forest Model

In [20]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

# Model Evaluation

In [21]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)


In [22]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,236,22
Actual 1,16,258


Accuracy Score : 0.9285714285714286
Classification Report
              precision    recall  f1-score   support

           0       0.94      0.91      0.93       258
           1       0.92      0.94      0.93       274

    accuracy                           0.93       532
   macro avg       0.93      0.93      0.93       532
weighted avg       0.93      0.93      0.93       532



# Feature Importance

In [23]:
# get important features
importances = rf_model.feature_importances_

#sort by importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.263911543779023, 'total_score'),
 (0.13835752991607672, 'home_points_game'),
 (0.1342473648853736, 'visitor_points_game'),
 (0.1255574798839996, 'Over/Under'),
 (0.045813832590131484, 'visitor_assists'),
 (0.04003547512244925, 'home_assists'),
 (0.03865522060547239, 'visitor_totReb'),
 (0.03433152704005456, 'home_totReb'),
 (0.02429271093908601, 'visitor_blocks'),
 (0.022095700022961018, 'home_blocks'),
 (0.0035148697223629437, 'home_name_Boston Celtics'),
 (0.0033256972233217694, 'visitor_name_Washington Wizards'),
 (0.0033025154250490344, 'home_name_Charlotte Hornets'),
 (0.002999068207595925, 'home_name_Washington Wizards'),
 (0.002812466544895079, 'visitor_name_Cleveland Cavaliers'),
 (0.002785126991811563, 'home_name_Chicago Bulls'),
 (0.0027752342581710042, 'visitor_name_San Antonio Spurs'),
 (0.0027537170280764957, 'home_name_Portland Trail Blazers'),
 (0.0026784467704680744, 'visitor_name_New Orleans Pelicans'),
 (0.0026637195848113566, 'home_name_Indiana Pacers'),
 (0.0026

In [24]:
# # save the model to a pickle file
# sports_bet_model = "sports_bet_model.pkl"

# # save the scaler to a pickle file
# joblib.dump(scaler, "scaler.pkl")